In [1]:
import atoti as tt

Welcome to Atoti 0.8.10!

By using this community edition, you agree with the license available at https://docs.atoti.io/latest/eula.html.
Browse the official documentation at https://docs.atoti.io.
Join the community at https://www.atoti.io/register.

Atoti collects telemetry data, which is used to help understand how to improve the product.
If you don't wish to send usage data, you can request a trial license at https://www.atoti.io/evaluation-license-request.

You can hide this message by setting the `ATOTI_HIDE_EULA_MESSAGE` environment variable to True.


In [2]:
session = tt.Session(
    user_content_storage=".content",
    port=9095,
    java_options=["-Xms1G", "-Xmx10G"]
)

In [3]:
db_name = "Olympic"
db_user = "postgres"
db_password = "postgres"  
db_host = "localhost"  
db_port = "5432"

In [4]:
jdbc_url = f"jdbc:postgresql://{db_host}:{db_port}/{db_name}?user={db_user}&password={db_password}"

In [5]:
jdbc_url

'jdbc:postgresql://localhost:5432/Olympic?user=postgres&password=postgres'

In [6]:
facttest = session.read_sql(
    "SELECT * FROM factolympic",
    keys=["year","country"],
    table_name="factTable",
    url=jdbc_url,
)

In [7]:
# medals = session.read_sql(
#     "SELECT * FROM medalsnew",
#     keys=["id"],
#     table_name="Medals",
#     url=jdbc_url)

In [8]:
countries = session.read_sql(
    "SELECT * FROM countries",
    keys=["country"],
    table_name="Countries",
    url=jdbc_url)

In [9]:
dimlife = session.read_sql(
    "SELECT * FROM dimlife",
    keys=["year","country"],
    table_name="Life",
    url=jdbc_url)

In [10]:
time = session.read_sql(
    "SELECT * FROM timenew",
    keys=["year"],
    table_name="Time",
    url=jdbc_url)

In [11]:
lifeexp = session.read_sql(
    "SELECT * FROM lifeexp",
    keys=["year","country"],
    table_name="LifeExp",
    url=jdbc_url)

In [12]:
facttest.join(countries, facttest["country"] == countries["country"])

In [13]:
facttest.join(dimlife, (facttest["year"] == dimlife["year"] )& (facttest["country"] == dimlife["country"]))

In [14]:
facttest.join(time, facttest["year"] == time["year"])

In [114]:
facttest.join(lifeexp, (facttest["year"] == lifeexp["year"] )& (facttest["country"] == lifeexp["country"]))

In [15]:
session.tables.schema

```mermaid
erDiagram
  "LifeExp" {
    _ String PK "country"
    _ String "code"
    _ int PK "year"
    nullable double "lifeexpectancy"
  }
  "Countries" {
    _ String PK "country"
    _ String "continent"
  }
  "Life" {
    _ int PK "year"
    _ String PK "country"
    _ String "lifeexpectancy"
    _ String "gdpcap"
  }
  "Time" {
    _ int PK "year"
    _ String "decade"
    _ String "season"
  }
  "factTable" {
    _ int PK "year"
    _ String PK "country"
    nullable int "bronze"
    nullable int "silver"
    nullable int "gold"
    nullable int "total"
    nullable double "depression"
    nullable double "schizophrenia"
    nullable double "bipolar"
    nullable double "eatingdisorder"
    nullable double "anxiety"
  }
  "factTable" }o--o| "Countries" : "`country` == `country`"
  "factTable" }o--o| "Life" : "(`country` == `country`) & (`year` == `year`)"
  "factTable" }o--o| "Time" : "`year` == `year`"
```


In [16]:
testcube = session.create_cube(facttest)

In [17]:
testcube

In [18]:
hierarchies, levels, measures = testcube.hierarchies, testcube.levels, testcube.measures

In [34]:
measures

{'contributors.COUNT': <atoti.measure.Measure object at 0x7f461a401660>, 'update.TIMESTAMP': <atoti.measure.Measure object at 0x7f461a402470>, 'total.SUM': <atoti.measure.Measure object at 0x7f461a403850>, 'total.MEAN': <atoti.measure.Measure object at 0x7f461a4013f0>, 'eatingdisorder.SUM': <atoti.measure.Measure object at 0x7f461a4032b0>, 'eatingdisorder.MEAN': <atoti.measure.Measure object at 0x7f461a400130>, 'gold.SUM': <atoti.measure.Measure object at 0x7f461a401900>, 'gold.MEAN': <atoti.measure.Measure object at 0x7f461a401990>, 'bipolar.SUM': <atoti.measure.Measure object at 0x7f461a402080>, 'bipolar.MEAN': <atoti.measure.Measure object at 0x7f461a403eb0>, 'depression.SUM': <atoti.measure.Measure object at 0x7f461a4023b0>, 'depression.MEAN': <atoti.measure.Measure object at 0x7f461a403070>, 'anxiety.SUM': <atoti.measure.Measure object at 0x7f461a4026b0>, 'anxiety.MEAN': <atoti.measure.Measure object at 0x7f461a401d50>, 'bronze.SUM': <atoti.measure.Measure object at 0x7f461a401180>, 'bronze.MEAN': <atoti.measure.Measure object at 0x7f461a402b60>, 'silver.SUM': <atoti.measure.Measure object at 0x7f461a402590>, 'silver.MEAN': <atoti.measure.Measure object at 0x7f461a402da0>, 'schizophrenia.SUM': <atoti.measure.Measure object at 0x7f461a4038e0>, 'schizophrenia.MEAN': <atoti.measure.Measure object at 0x7f461a402140>}

In [32]:
list(levels)

[('Countries', 'continent', 'continent'),
 ('Countries', 'continent', 'country'),
 ('Details', 'details', 'lifeexpectancy'),
 ('Details', 'details', 'gdpcap'),
 ('Time', 'time', 'decade'),
 ('Time', 'time', 'year'),
 ('Time', 'time', 'season')]

In [23]:
hierarchies["Time","time"] = [time["decade"],time["year"],time["season"]]

In [31]:
del hierarchies["Life","gdpcap"]

In [38]:
list(testcube.hierarchies)

[('Countries', 'continent'), ('Details', 'details'), ('Time', 'time')]

In [45]:
testcube.query(measures["total.SUM"], levels=[levels[('Countries', 'continent', 'country')],levels["Details","details","lifeexpectancy"]])

total.SUM
continent     country      lifeexpectancy          
Africa        Algeria      N/A                   15
              Angola       N/A                    0
              Benin        N/A                    0
              Botswana     N/A                    1
              Burkina Faso N/A                    0
...                                             ...
South America Paraguay     N/A                    1
              Peru         N/A                    1
              Suriname     N/A                    1
              Uruguay      N/A                    1
              Venezuela    N/A                    7

[198 rows x 1 columns]

lifeexpectancy gdpcap
year country                                   
2020 Poland                         High   High
     Guinea                       Medium    Low
     South Sudan                     Low    N/A
     Greece                         High   High
     United Arab Emirates           High   High